In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib inline
## import 부분입니다
df=pd.DataFrame()
list=os.listdir('./df_total/')
for list in list:
    df1= pd.read_csv('./df_total/{}'.format(list))
    df=pd.concat([df,df1])


In [40]:
df = df.dropna(subset=['macc_x_mean', 'mgps_lat_std', 'e4acc_x_mean', 'temp_mean', 'bvp_std', 'hr_std'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['ts', 'action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity'], axis=1)
y = df['action']


## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 300)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))

정확도 : 0.8570016362148896


In [41]:
lgbm_wrapper = LGBMClassifier(n_estimators=100)
evals=[(X_test,y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=1000, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)
## n_ estimator 가 200을 넘어가면 오히려 loss 가 증가하는 경향을 보여서 100으로 설정
def get_clf_eval(y_test,pred = None,pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = round(precision_score(y_test, pred, average='micro'), ndigits=3)
    recall = round(recall_score(y_test, pred, average='micro'), ndigits=3)
    f1 = round(f1_score(y_test, pred, average='micro'), ndigits=3)
    print('정확도:{0:,f},정밀도:{1:f},\
    재현율:{2:f},f1:{3:f}'.format(accuracy,precision,recall,f1),'\n')
# 모델 평가 지표 함수
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]
get_clf_eval(y_test,preds,pred_proba)

c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 1.39791
[2]	valid_0's multi_logloss: 1.27324
[3]	valid_0's multi_logloss: 1.17538
[4]	valid_0's multi_logloss: 1.10901
[5]	valid_0's multi_logloss: 1.04925
[6]	valid_0's multi_logloss: 1.0014
[7]	valid_0's multi_logloss: 0.962577
[8]	valid_0's multi_logloss: 0.928847
[9]	valid_0's multi_logloss: 0.900186
[10]	valid_0's multi_logloss: 0.876551
[11]	valid_0's multi_logloss: 0.857193
[12]	valid_0's multi_logloss: 0.836384
[13]	valid_0's multi_logloss: 0.821551
[14]	valid_0's multi_logloss: 0.804983
[15]	valid_0's multi_logloss: 0.794411
[16]	valid_0's multi_logloss: 0.781348
[17]	valid_0's multi_logloss: 0.773966
[18]	valid_0's multi_logloss: 0.764362
[19]	valid_0's multi_logloss: 0.753948
[20]	valid_0's multi_logloss: 0.744613
[21]	valid_0's multi_logloss: 0.732051
[22]	valid_0's multi_logloss: 0.724366
[23]	valid_0's multi_logloss: 0.719677
[24]	valid_0's multi_logloss: 0.717006
[25]	valid_0's multi_logloss: 0.714011
[26]	valid_0's multi_logloss: 0.700583
[2

In [42]:
df=pd.DataFrame()
list=os.listdir('./df_total/')
for list in list:
    df1= pd.read_csv('./df_total/{}'.format(list))
    df=pd.concat([df,df1])


In [44]:
## e4 결측치를 해당 ts m값으로 대체 - 그 후에도 결측치가 존재할경우 drop
df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']] = df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']].fillna(df[['macc_x_mean','macc_y_mean','macc_z_mean']])
df = df.dropna(subset=['macc_x_mean', 'mgps_lat_std', 'e4acc_x_mean', 'temp_mean', 'bvp_std', 'hr_std'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['ts', 'action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity'], axis=1)
y = df['action']


## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 300)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))


정확도 : 0.8545814016907554


In [45]:
df=pd.DataFrame()
list=os.listdir('./To/df_total_2019/')
for list in list:
    df1= pd.read_csv('./To/df_total_2019/{}'.format(list))
    df=pd.concat([df,df1])

In [46]:


df = df.dropna(subset=['macc_x_mean', 'mgps_lat_std', 'e4acc_x_mean', 'temp_mean', 'bvp_std', 'hr_std'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['ts', 'action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity'], axis=1)
y = df['action']


In [50]:


## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 250)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))

정확도 : 0.810126582278481


In [51]:
lgbm_wrapper = LGBMClassifier(n_estimators=100)
evals=[(X_test,y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=1000, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)
## n_ estimator 가 200을 넘어가면 오히려 loss 가 증가하는 경향을 보여서 100으로 설정
def get_clf_eval(y_test,pred = None,pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = round(precision_score(y_test, pred, average='micro'), ndigits=3)
    recall = round(recall_score(y_test, pred, average='micro'), ndigits=3)
    f1 = round(f1_score(y_test, pred, average='micro'), ndigits=3)
    print('정확도:{0:,f},정밀도:{1:f},\
    재현율:{2:f},f1:{3:f}'.format(accuracy,precision,recall,f1),'\n')
# 모델 평가 지표 함수
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]
get_clf_eval(y_test,preds,pred_proba)

c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 1.4742
[2]	valid_0's multi_logloss: 1.31578
[3]	valid_0's multi_logloss: 1.19778
[4]	valid_0's multi_logloss: 1.10438
[5]	valid_0's multi_logloss: 1.03645
[6]	valid_0's multi_logloss: 0.976351
[7]	valid_0's multi_logloss: 0.91834
[8]	valid_0's multi_logloss: 0.873231
[9]	valid_0's multi_logloss: 0.837815
[10]	valid_0's multi_logloss: 0.797401
[11]	valid_0's multi_logloss: 0.764832
[12]	valid_0's multi_logloss: 0.741291
[13]	valid_0's multi_logloss: 0.72208
[14]	valid_0's multi_logloss: 0.706463
[15]	valid_0's multi_logloss: 0.689997
[16]	valid_0's multi_logloss: 0.674885
[17]	valid_0's multi_logloss: 0.659148
[18]	valid_0's multi_logloss: 0.651343
[19]	valid_0's multi_logloss: 0.643734
[20]	valid_0's multi_logloss: 0.640057
[21]	valid_0's multi_logloss: 0.627941
[22]	valid_0's multi_logloss: 0.624075
[23]	valid_0's multi_logloss: 0.618487
[24]	valid_0's multi_logloss: 0.613318
[25]	valid_0's multi_logloss: 0.608417
[26]	valid_0's multi_logloss: 0.602764
[27

In [59]:
df=pd.DataFrame()
list=os.listdir('./To/df_total_2019/')
for list in list:
    df1= pd.read_csv('./To/df_total_2019/{}'.format(list))
    df=pd.concat([df,df1])
df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']] = df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']].fillna(df[['macc_x_mean','macc_y_mean','macc_z_mean']])

df = df.dropna(subset=['macc_x_mean', 'mgps_lat_std', 'e4acc_x_mean', 'temp_mean', 'bvp_std', 'hr_std'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['ts', 'action', 'actionOption', 'actionSub', 'actionSubOption', 'condition', 'conditionSub1Option', 'conditionSub2Option', 'place', 'emotionPositive', 'emotionTension', 'activity'], axis=1)
y = df['action']



## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 400)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))

정확도 : 0.8438818565400844


In [60]:
lgbm_wrapper = LGBMClassifier(n_estimators=100)
evals=[(X_test,y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=1000, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)
## n_ estimator 가 200을 넘어가면 오히려 loss 가 증가하는 경향을 보여서 100으로 설정
def get_clf_eval(y_test,pred = None,pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = round(precision_score(y_test, pred, average='micro'), ndigits=3)
    recall = round(recall_score(y_test, pred, average='micro'), ndigits=3)
    f1 = round(f1_score(y_test, pred, average='micro'), ndigits=3)
    print('정확도:{0:,f},정밀도:{1:f},\
    재현율:{2:f},f1:{3:f}'.format(accuracy,precision,recall,f1),'\n')
# 모델 평가 지표 함수
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]
get_clf_eval(y_test,preds,pred_proba)

c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 1.41061
[2]	valid_0's multi_logloss: 1.25579
[3]	valid_0's multi_logloss: 1.1418
[4]	valid_0's multi_logloss: 1.05554
[5]	valid_0's multi_logloss: 0.98353
[6]	valid_0's multi_logloss: 0.922603
[7]	valid_0's multi_logloss: 0.870346
[8]	valid_0's multi_logloss: 0.818289
[9]	valid_0's multi_logloss: 0.773852
[10]	valid_0's multi_logloss: 0.74068
[11]	valid_0's multi_logloss: 0.712551
[12]	valid_0's multi_logloss: 0.679153
[13]	valid_0's multi_logloss: 0.658189
[14]	valid_0's multi_logloss: 0.638774
[15]	valid_0's multi_logloss: 0.622474
[16]	valid_0's multi_logloss: 0.605809
[17]	valid_0's multi_logloss: 0.589948
[18]	valid_0's multi_logloss: 0.580141
[19]	valid_0's multi_logloss: 0.567997
[20]	valid_0's multi_logloss: 0.557791
[21]	valid_0's multi_logloss: 0.552671
[22]	valid_0's multi_logloss: 0.548785
[23]	valid_0's multi_logloss: 0.542141
[24]	valid_0's multi_logloss: 0.539731
[25]	valid_0's multi_logloss: 0.536055
[26]	valid_0's multi_logloss: 0.529819
[2

In [72]:
df=pd.DataFrame()
list=os.listdir('./total/')
for list in list:
    df1= pd.read_csv('./total/{}'.format(list))
    df1=df1[['action','macc_x_mean','e4acc_x_mean','macc_y_mean','e4acc_y_mean','macc_z_mean','e4acc_z_mean']]
    df=pd.concat([df,df1])

In [79]:
df = df.dropna(subset=['macc_x_mean', 'e4acc_x_mean'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['action'], axis=1)
y = df['action']


## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 400)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))

정확도 : 0.7447273162413618


In [82]:
lgbm_wrapper = LGBMClassifier(n_estimators=1000)
evals=[(X_test,y_test)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=1000, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_test)
## n_ estimator 가 200을 넘어가면 오히려 loss 가 증가하는 경향을 보여서 100으로 설정
def get_clf_eval(y_test,pred = None,pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = round(precision_score(y_test, pred, average='micro'), ndigits=3)
    recall = round(recall_score(y_test, pred, average='micro'), ndigits=3)
    f1 = round(f1_score(y_test, pred, average='micro'), ndigits=3)
    print('정확도:{0:,f},정밀도:{1:f},\
    재현율:{2:f},f1:{3:f}'.format(accuracy,precision,recall,f1),'\n')
# 모델 평가 지표 함수
pred_proba = lgbm_wrapper.predict_proba(X_test)[:,1]
get_clf_eval(y_test,preds,pred_proba)

c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\jhm21\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 1.53571
[2]	valid_0's multi_logloss: 1.45628
[3]	valid_0's multi_logloss: 1.40041
[4]	valid_0's multi_logloss: 1.35754
[5]	valid_0's multi_logloss: 1.32149
[6]	valid_0's multi_logloss: 1.29297
[7]	valid_0's multi_logloss: 1.26821
[8]	valid_0's multi_logloss: 1.24621
[9]	valid_0's multi_logloss: 1.2272
[10]	valid_0's multi_logloss: 1.21048
[11]	valid_0's multi_logloss: 1.19671
[12]	valid_0's multi_logloss: 1.18306
[13]	valid_0's multi_logloss: 1.17107
[14]	valid_0's multi_logloss: 1.16142
[15]	valid_0's multi_logloss: 1.15043
[16]	valid_0's multi_logloss: 1.14083
[17]	valid_0's multi_logloss: 1.13232
[18]	valid_0's multi_logloss: 1.12596
[19]	valid_0's multi_logloss: 1.11842
[20]	valid_0's multi_logloss: 1.11084
[21]	valid_0's multi_logloss: 1.10444
[22]	valid_0's multi_logloss: 1.09848
[23]	valid_0's multi_logloss: 1.09321
[24]	valid_0's multi_logloss: 1.08742
[25]	valid_0's multi_logloss: 1.08226
[26]	valid_0's multi_logloss: 1.07768
[27]	valid_0's multi_l

In [83]:
df=pd.DataFrame()
list=os.listdir('./total/')
for list in list:
    df1= pd.read_csv('./total/{}'.format(list))
    df1=df1[['action','macc_x_mean','e4acc_x_mean','macc_y_mean','e4acc_y_mean','macc_z_mean','e4acc_z_mean']]
    df=pd.concat([df,df1])
df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']] = df[['e4acc_x_mean','e4acc_y_mean','e4acc_z_mean']].fillna(df[['macc_x_mean','macc_y_mean','macc_z_mean']])

df = df.dropna(subset=['macc_x_mean', 'e4acc_x_mean'], axis=0)
df = df[(df['action']!='sleep')&(df['action']!='recreation_etc')&(df['action']!='meal')&(df['action']!='hobby')]
X = df.drop(['action'], axis=1)
y = df['action']


## sleep은 추가적인 자료에 있고, 측정이 어렵기에 제거, recreation etc는 행위의 종류가 다양하여 제거 , 식사도 측정이 불가하다고 판단
## 취미또한 한 테마안에 여러 활동이 포함되어 제거
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)


## 수민님이 주신 scaler
for col in X_train.columns:
    scal = RobustScaler()
    X_train[col] = scal.fit_transform(np.array(X_train[col]).reshape(-1, 1))
    X_test[col] = scal.transform(np.array(X_test[col]).reshape(-1, 1))

classifier = RandomForestClassifier(n_estimators = 400)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("정확도 : {}".format(accuracy_score(y_test, y_pred)))


정확도 : 0.7473898345648727
